# Poker Probabilities

Driven by a desire to develop a statistical mental framework in late night online poker sessions, I thought it may be fun to create a probability calculator for the game. This code's desired state is to allow the user to input their hand and current board, so the calculator can return the probabilities of getting a pair, two_pair, straight, flush, etc. The challenge here is to understand different combinations and permutations of possible hands, given current inputs, to then code up the functions to calculate probabilities.
Unfortunately, this project remains unfinished, where all the wrapper classes are generated, but only the flush calculation is coded. 

Challenges I have faced include:
- Figuring out how to verify my maths is correct
- Evaluating which combinations/permutations are needed for each possibility

## Usage
Given your hand and the current board, this script will calculate the probabilities of:
- one pair
- two pair
- three of a kind
- four of a kind
- a full house
- a flush
- a straght

### Initialise:
`p = PokerCalculator((6h, 7h), (5c, Js, Ad))`

This means:
- Your hand is a 6 of Hearts and 7 of Hearts
- The board is a 5 of Clubs, Jack of Spades and Ace of Diamonds

`p.calc_flush()`
> 0 

`p.calc_full_house()`
> 0

`p.calc_all()`
- > Three of a Kind: 0.3 
- > Four of a Kind: 0.12
- > Flush: 0.
...

In [70]:
import random
from math import comb as c
from math import factorial as f

In [64]:
class PokerCalculator:
    def __init__(self, hand:list, board: list = []):
        self.board = board
        self.hand = hand
        self.cons = self.hand + self.board # consolidated hand
        self.suits = sorted([s for _, s in self.cons])
        self.nums = sorted([n for n, _ in self.cons])
        # self.deck = self.create_deck()
        self.remaining = 7 - len(self.cons)
        self.revealed = len(self.cons)
        

    def create_deck(self) -> list:
        """Creates a deck of cards to aid simulation in code"""
        deck = []
        for num in ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'A', 'J', 'Q', 'K']: 
            for suit in ["h", "d", "s", "c"]:
                deck.append(num+suit)
        for card in self.cons:
            deck.remove(card)
        return deck

    def is_flush(self) -> bool:
        """Evaluates whether the board+hand has a flush"""
        if not self.board:
            return False
        ct = 1
        # o(n)
        for i in range(len(self.suits)):
            if ct == 5:
                return True
            if self.suits[i] == self.suits[i-1]:
                ct += 1
            else:
                ct = 1
        return False
    
    def is_one_pair(self):
        """Evaluates whether the board+hand is a one-pair"""
        for num in self.nums:
            if self.nums.count(num) == 2:
                print(num)
                return True
        return False

    def calc_one_pair(self):
        """Evaluates the probability of getting a one_pair given the board and hand
        Note: This function is currently incorrect"""
        if self.is_one_pair():
            return 1
        
        n_ways = 0
        for num in self.nums:
            have = self.nums.count(num)
            non_num = 52 - self.revealed - 4 + have
            n_ways += c(4-have,1) * c(non_num, self.remaining-1)
        
        return n_ways / c(52-self.revealed, self.remaining)

    def calc_flush(self):
        """ Evaluates the probability of attaining a flush given the current board+hand
        """
        if self.is_flush():
            return 1
        
        n_ways = 0
        for suit in set(self.suits):
            have = self.suits.count(suit)  
            non_suit = 52 - self.revealed - 13 + have
            choose = self.remaining
            while choose + have >= 5:
                n_ways += (c(13-have, choose)*c(non_suit, self.remaining-choose))
                choose -= 1
        
        return n_ways / c(52-self.revealed, self.remaining)

    def print_all(self):
        """Prints the current board+hand in a human readable format"""
        print(f"Hand: {self.hand}, Board: {self.board}")
        

In [75]:
pc = PokerCalculator(['7h', '6h'], [])

pc.calc_flush()

0.06399828201400819

## Automated Testing

This class simulates picking from a deck to generate possible hand + board combinations. Using these hand+board combinations, we can initialise the `PokerCalculator` object and perform testing. 

In [54]:
# Hand + Board Generator
class PokerGenerator():
    def __init__(self):
        self.deck = self.create_deck()

    def create_deck(self):
        deck = []
        for num in ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'A', 'J', 'Q', 'K']: 
            for suit in ["h", "d", "s", "c"]:
                deck.append(num+suit)
        return deck

    def choice(self) -> str:
        card = random.choice(self.deck)
        self.deck.remove(card)
        return card

    def generate_hand(self):
        return [self.choice(), self.choice()]

    def generate_board(self, length = None):
        if not length:
            length = random.choice([0,3,4,5])
        return list([self.choice() for _ in range(length)])

    def generate_object(self, board_size = None):
        hand = self.generate_hand()
        board = self.generate_board(board_size)
        p = PokerCalculator(hand, board)
        return p

In [76]:
# Testing if is_one_pair works (Manual Checking)
for _ in range(2000):
    pg = PokerGenerator()
    p = pg.generate_object()
    p.print_all()
    print(p.is_one_pair())

Hand: ('9s', '7c'), River: ('9c', 'Jd', '2d')
9
True
Hand: ('2c', 'Jd'), River: ('4c', '6d', 'Qc')
False
Hand: ('4s', 'Ks'), River: ()
False
Hand: ('5h', 'Qs'), River: ('2s', '4d', '2c', '6d')
2
True
Hand: ('6h', '4h'), River: ('5c', 'Th', 'As', 'Kd', '5s')
5
True
Hand: ('Qh', 'Kc'), River: ()
False
Hand: ('Kh', '8s'), River: ('Kc', '3d', '6c', 'Th', '7s')
K
True
Hand: ('5s', 'Qd'), River: ('Ac', 'Js', '3h')
False
Hand: ('6d', 'Ac'), River: ('8c', 'Ah', 'Ks', 'Qc', 'Kh')
A
True
Hand: ('9h', '2d'), River: ('4h', '9c', '7c', '4d')
4
True
Hand: ('8d', '9d'), River: ('8h', '5c', 'Ad')
8
True
Hand: ('Jd', '6h'), River: ()
False
Hand: ('4d', '4c'), River: ('Th', 'Jh', '3d')
4
True
Hand: ('Jh', 'Th'), River: ()
False
Hand: ('Jd', '7d'), River: ('2h', '3d', '9d', 'Qs')
False
Hand: ('4c', 'Qc'), River: ()
False
Hand: ('2s', '8c'), River: ('Qc', '8s', '7s', 'Ts')
8
True
Hand: ('5c', '8h'), River: ()
False
Hand: ('Qc', '5h'), River: ('Js', 'Qh', '3h', 'Ts', 'Ah')
Q
True
Hand: ('Td', '7s'), River: